In [ ]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [7]:
import os

import cudf

import nvtabular as nvt

In [8]:
from merlin_models.tensorflow.models.retrieval import YouTubeDNN

In [10]:
INPUT_DATA_DIR = os.environ.get(
    "INPUT_DATA_DIR", os.path.expanduser("~/nvt-examples/end-to-end-poc/data/")
)

In [11]:
examples = cudf.read_parquet(os.path.join(INPUT_DATA_DIR, "grouped_examples.parquet"))
examples.head()

sampled_tag  \
userId day                                                       
1      4146  [43870, 30243, 31462, 25871, 28458, 44763, 371...   
2      4071  [47665, 65105, 38902, 64349, 1452, 57559, 6574...   
3      7521  [21601, 35319, 30185, 10741, 52728, 39667, 661...   
       7688  [28185, 51431, 45744, 50807, 67439, 56292, 467...   
       8045  [15910, 13153, 52882, 41183, 63025, 72409, 107...   

                                                     timestamp  \
userId day                                                       
1      4146  [1147868053, 1147868097, 1147868414, 114786846...   
2      4071  [1141415528, 1141415566, 1141415576, 114141558...   
3      7521  [1439472199, 1439472203, 1439472211, 143947221...   
       7688  [1453904021, 1453904031, 1453904046, 145390404...   
       8045  [1484753654, 1484753766, 1484753808, 148475384...   

                                                       movieId  movieId_count  \
userId day                                                                      
1      4146  [5952, 1653, 1250, 6539, 6377, 3448, 1088, 899...             52   
2      4071  [5952, 497, 1374, 1653, 2640, 5445, 151, 236, ...            124   
3      7521  [356, 593, 1270, 1, 2571, 260, 318, 1196, 527,...            178   
       7688  [1206, 1208, 44191, 32587, 40815, 36529, 45186...              9   
       8045  [1089, 4011, 741, 778, 111, 214, 293, 1252, 33...             21   

             target_item                                              genre  
userId day                                                                   
1      4146         7361  [6, 6, 3, 6, 17, 9, 16, 3, 9, 2, 3, 6, 10, 3, ...  
2      4071         2150  [6, 6, 6, 6, 6, 9, 9, 7, 9, 15, 7, 9, 2, 6, 6,...  
3      7521        37729  [6, 9, 16, 9, 2, 6, 7, 9, 17, 3, 4, 5, 6, 10, ...  
       7688         4344  [12, 12, 18, 7, 9, 18, 7, 9, 6, 9, 1, 1, 12, 2...  
       8045        27773  [7, 9, 17, 18, 7, 9, 18, 7, 9, 9, 7, 9, 9, 18,...

In [ ]:
# TODO: Build feature columns for the input features

## Hyper-parameters

In [ ]:
BATCH_SIZE = 1024  # Batch Size
CATEGORICAL_COLUMNS = []  # Single-hot
CATEGORICAL_MH_COLUMNS = ["sampled_tag", "movieId", "genre"]  # Multi-hot
NUMERIC_COLUMNS = ["movieId_count"]

In [ ]:
# TODO: Sort out how to get embedding table shapes from workflow

In [ ]:
# workflow = nvt.Workflow.load(os.path.join(INPUT_DATA_DIR, "workflow"))

In [ ]:
# EMBEDDING_TABLE_SHAPES, MH_EMBEDDING_TABLE_SHAPES = nvt.ops.get_embedding_sizes(workflow)
# EMBEDDING_TABLE_SHAPES.update(MH_EMBEDDING_TABLE_SHAPES)
# EMBEDDING_TABLE_SHAPES

## DataLoader

In [ ]:
# TODO: Construct a KSL DataLoader to load the data

In [ ]:
import os
import tensorflow as tf

# we can control how much memory to give tensorflow with this environment variable
# IMPORTANT: make sure you do this before you initialize TF's runtime, otherwise
# TF will have claimed all free GPU memory
os.environ["TF_MEMORY_ALLOCATION"] = "0.7"  # fraction of free memory
from nvtabular.loader.tensorflow import KerasSequenceLoader, KerasSequenceValidater

In [ ]:
train_dataset_tf = KerasSequenceLoader(
    TRAIN_PATHS,  # you could also use a glob pattern
    batch_size=BATCH_SIZE,
    label_names=["rating"],
    cat_names=CATEGORICAL_COLUMNS + CATEGORICAL_MH_COLUMNS,
    cont_names=NUMERIC_COLUMNS,
    engine="parquet",
    shuffle=True,
    buffer_size=0.06,  # how many batches to load at once
    parts_per_chunk=1,
)

In [ ]:
batch = next(iter(train_dataset_tf))
batch[0]

In [ ]:
inputs = {}  # tf.keras.Input placeholders for each feature to be used
emb_layers = []  # output of all embedding layers, which will be concatenated

In [ ]:
for col in CATEGORICAL_COLUMNS:
    inputs[col] = tf.keras.Input(name=col, dtype=tf.int32, shape=(1,))
# Note that we need two input tensors for multi-hot categorical features
for col in CATEGORICAL_MH_COLUMNS:
    inputs[col + "__values"] = tf.keras.Input(name=f"{col}__values", dtype=tf.int64, shape=(1,))
    inputs[col + "__nnzs"] = tf.keras.Input(name=f"{col}__nnzs", dtype=tf.int64, shape=(1,))

In [ ]:
for col in CATEGORICAL_COLUMNS + CATEGORICAL_MH_COLUMNS:
    emb_layers.append(
        tf.feature_column.embedding_column(
            tf.feature_column.categorical_column_with_identity(
                col, EMBEDDING_TABLE_SHAPES[col][0]
            ),  # Input dimension (vocab size)
            EMBEDDING_TABLE_SHAPES[col][1],  # Embedding output dimension
        )
    )
emb_layers

In [ ]:
# TODO: Add optimizer and other training config

In [ ]:
model = tf.keras.Model(inputs=inputs, outputs=x)
model.compile("sgd", "binary_crossentropy")

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
# TODO: Train the model!

In [ ]:
validation_callback = KerasSequenceValidater(valid_dataset_tf)

history = model.fit(train_dataset_tf, callbacks=[validation_callback], epochs=1)

In [ ]:
MODEL_NAME_TF = os.environ.get("MODEL_NAME_TF", "movielens_tf")
MODEL_PATH_TEMP_TF = os.path.join(MODEL_BASE_DIR, MODEL_NAME_TF, "1/model.savedmodel")

model.save(MODEL_PATH_TEMP_TF)